In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(9211, 128, 128, 3)

In [8]:
y.shape

(9211, 5)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(7368, 128, 128, 3)

In [11]:
X_test.shape

(1843, 128, 128, 3)

In [12]:
y_train.shape

(7368, 5)

In [13]:
y_test.shape

(1843, 5)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

5

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

2023-05-31 16:51:41.229532: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10
231/231 [==============================] - 79s 332ms/step - loss: 0.3726 - accuracy: 0.8715 - val_loss: 0.4245 - val_accuracy: 0.8573
Epoch 2/10
231/231 [==============================] - 97s 419ms/step - loss: 0.1749 - accuracy: 0.9369 - val_loss: 0.1850 - val_accuracy: 0.9300
Epoch 3/10
231/231 [==============================] - 93s 400ms/step - loss: 0.1137 - accuracy: 0.9591 - val_loss: 0.0699 - val_accuracy: 0.9756
Epoch 4/10
231/231 [==============================] - 94s 408ms/step - loss: 0.0874 - accuracy: 0.9712 - val_loss: 0.0845 - val_accuracy: 0.9718
Epoch 5/10
231/231 [==============================] - 93s 403ms/step - loss: 0.0764 - accuracy: 0.9760 - val_loss: 0.1369 - val_accuracy: 0.9495
Epoch 6/10
231/231 [==============================] - 98s 424ms/step - loss: 0.0665 - accuracy: 0.9761 - val_loss: 0.0989 - val_accuracy: 0.9696
Epoch 7/10
231/231 [==============================] - 96s 416ms/step - loss: 0.0480 - accuracy: 0.9836 - val_loss: 0.0962 - val_ac

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 90.94%


In [19]:
cnn_scores

[0.24874261021614075, 0.9093868732452393]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

58/58 [==============================] - 6s 101ms/step
Accuracy: 90.94%
Precision: 92.74%
Recall: 90.94%
F1 score: 90.52%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-cerry+kentang.csv', index=False)

+------+-----------------------------------------------------------------------+------------------------------------------+------------------------------------------+
|      | Filename                                                              | True Labels                              | Predicted Labels                         |
|------+-----------------------------------------------------------------------+------------------------------------------+------------------------------------------|
|    0 | 542c1489-0040-4867-acb3-82f73249e2f8___RS_LB 4592_180deg.JPG          | Potato___Late_blight                     | Potato___Late_blight                     |
|    1 | 60e9a878-13e0-4345-964c-fd807cd20bbb___RS_HL 1876_flipTB.JPG          | Potato___healthy                         | Potato___healthy                         |
|    2 | b255bae1-b650-413d-9444-dff61de6bd33___JR_HL 9724_flipTB.JPG          | Cherry_(including_sour)___healthy        | Potato___healthy                         

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-cerry+kentang.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------+-----------------------------------+------------------------------------------+
|      | Filename                                                             | True Labels                       | Predicted Labels                         |
|------+----------------------------------------------------------------------+-----------------------------------+------------------------------------------|
|    2 | b255bae1-b650-413d-9444-dff61de6bd33___JR_HL 9724_flipTB.JPG         | Cherry_(including_sour)___healthy | Potato___healthy                         |
|   18 | b87b1766-d629-4bfb-b9d3-6d5fe336cfef___JR_HL 4182.JPG                | Cherry_(including_sour)___healthy | Potato___healthy                         |
|   52 | 515f8058-9e91-4b9a-b74a-deb114fc3f39___JR_HL 9444.JPG                | Cherry_(including_sour)___healthy | Potato___healthy                         |
|   61 | 9f57292a-576b-4ae8

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

231/231 [==============================] - 25s 109ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 168.53584694862366 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

58/58 [==============================] - 8s 132ms/step
Random Forest Classifier Accuracy: 98.86%
Random Forest Classifier Precision: 0.99
Random Forest Classifier Recall: 0.99
Random Forest Classifier F1 Score: 0.99


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-cerry+kentang.csv', index=False)

+------+-----------------------------------------------------------------------+------------------------------------------+------------------------------------------+
|      | Filename                                                              | True Labels                              | Predicted Labels                         |
|------+-----------------------------------------------------------------------+------------------------------------------+------------------------------------------|
|    0 | 542c1489-0040-4867-acb3-82f73249e2f8___RS_LB 4592_180deg.JPG          | Potato___Late_blight                     | Potato___Late_blight                     |
|    1 | 60e9a878-13e0-4345-964c-fd807cd20bbb___RS_HL 1876_flipTB.JPG          | Potato___healthy                         | Potato___healthy                         |
|    2 | b255bae1-b650-413d-9444-dff61de6bd33___JR_HL 9724_flipTB.JPG          | Cherry_(including_sour)___healthy        | Cherry_(including_sour)___healthy        

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-cerry+kentang.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------+------------------------------------------+-----------------------------------+
|      | Filename                                                             | True Labels                              | Predicted Labels                  |
|------+----------------------------------------------------------------------+------------------------------------------+-----------------------------------|
|   82 | 47c3230c-dc1e-4787-a533-2b5afc0cdffa___JR_HL 9491.JPG                | Cherry_(including_sour)___healthy        | Potato___Early_blight             |
|  207 | 14484ea5-9c34-427c-abe4-2743265aced0___RS_HL 4204_new90degFlipTB.JPG | Potato___healthy                         | Potato___Late_blight              |
|  468 | ce5ec29b-599c-4c69-ba4d-04df7ab1c46f___RS_LB 4945_180deg.JPG         | Potato___Late_blight                     | Potato___Early_blight             |
|  548 | 5a3fc4bb-a78c-4276